In [1]:
import sys, os, time, math, random
import numpy as np
from tqdm import trange
import hickle as hkl

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize

import pycuda.autoinit
import pycuda.driver as cuda
import pycuda.gpuarray as gpua
import pycuda.curandom
from pycuda.compiler import SourceModule
from pycuda import cumath

import skcuda
import skcuda.linalg as linalg
linalg.init()

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

(x_train, y_train), (x_test, y_test) = hkl.load('flowers102.hkl')

# from sklearn.model_selection import train_test_split
# (x, y) = hkl.load('scene15_hsnn.hkl')
# x = x.T
# y = y.T

# x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.3)

C:\Users\chand\Anaconda3\lib\site-packages\skcuda\cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [4]:

# Matrix Inverse
def matrix_inverse(A, cuda = True, coefficient = None):
    m, n = A.shape
    transpose = False
    pinv = None
    
#     A is m-by-n and the rank of A is equal to m (m ≤ n) then A has right inverse A_P = At * inv(A * At)
#     A is m-by-n and the rank of A is equal to n (n ≤ m) then A has left inverse A_P = A * inv(At * A)
#     By default this calculates the right inverse
    if m > n:
        A = np.ascontiguousarray(A.T)
        transpose = True

    At = np.ascontiguousarray(A.T)
    if cuda:
#         copy the input matrix and its transpose into gpu memory
        A_gpu = gpua.to_gpu(A)
        At_gpu = gpua.to_gpu(At)
        
        if coefficient is None:
            inv = linalg.inv(linalg.dot(A_gpu, At_gpu))
        else:
            inv = linalg.inv( gpua.to_gpu(np.identity(A.shape[0]) / coefficient) +  linalg.dot(A_gpu, At_gpu))
            
        pinv = linalg.dot(At_gpu, inv).get()
    else:
        if coefficient is None:
            inv = np.linalg.inv(A.dot(At))
        else:
            inv = np.linalg.inv((np.identity(A.shape[0])/ coefficient) + A.dot(At))
        pinv = At.dot(inv)
        
    if transpose:
        pinv = pinv.T
        
    return pinv

In [5]:
import random
class ELM:
    def __init__(self, hidden_nodes, activation):
        self.features = None
        self.hidden_nodes = hidden_nodes
        self.activation = activation

        self.input_weights = None
        self.output_weights = None  # beta
        self.bias = None

    def hidden_layer(self, x):
        bias = np.array([self.bias[0],]* x.shape[0])
        h = np.dot(x, self.input_weights) + bias
        
        h = h + 0j
        
        if self.activation == 'relu':
            h = np.maximum(h, 0, h)
        elif self.activation == 'sigmoid':
            h = 1 / (1 + np.exp(-h))
        elif self.activation == 'sine':
            h = np.sin(h)
        return h.real

    def fit(self, x_train, y_train):
        #number of features for each sample in the input
        self.features = x_train.shape[1]
        #intialize random weights and bias
        self.input_weights = np.random.normal(size=(x_train.shape[1], self.hidden_nodes))
        self.bias = np.random.random((1, self.hidden_nodes))
        #calculates the dot product(representations) of the input data and created weights
        H = self.hidden_layer(x_train)
        #Calculates the output weights (Beta)
        self.output_weights = np.dot(np.linalg.pinv(H), y_train)

        pred = self.predict(x_train)
        acc = self.performance(y_train, pred)
        return pred, acc

    def predict(self, x):
        H = self.hidden_layer(x)
        predictions = np.dot(H, self.output_weights)
        return predictions

    def evaluate(self, x_test, y_test):
        pred = self.predict(x_test)
        acc = self.performance(y_test, pred)
        return pred, acc

    def performance(self, y_actual, y_predicted):
        y_actual = np.argmax(y_actual, axis=-1)
        y_predicted = np.argmax(y_predicted, axis=-1)

        correct = np.sum(y_actual == y_predicted)
        accuracy = (correct / y_actual.shape[0]) * 100
        accuracy = format(accuracy, '.2f')
        return accuracy

In [6]:

class MSNN():
    def __init__(self):
#         Stores the matrices from first layer
        self.af = []
        self.bf = []
#         Stores representation after each iteration
        self.hf = []
        
        self.scalers = []
        self.act_fn = None
    
    def exp(self, x):
        x_gpu = gpua.to_gpu(x)
        x_exp = cumath.exp(x_gpu).get()
        return x_exp
    
    def act(self, h):
        if self.act_fn == 'sigmoid':
            h = 1/ (1+ self.exp(-h))
        elif self.act_fn == 'sine':
            h = np.sin(h)
        return h

    def act_inv(self, h):
        if self.act_fn == 'sigmoid':
            h = -1 * np.log((1/h) -1)
        elif self.act_fn == 'sine':
            h = np.arcsin(h)
        return h
    
    #Trains the model
    def fit(self,x, y, d, l_max, act_fn, coefficient):
        self.act_fn = act_fn
        
        instances = x.shape[0] #number of samples
        features = x.shape[1] # number of features for each sample
        
        y_scaler = MinMaxScaler(feature_range=(0.0001, 0.9999))
        p_scaler = MinMaxScaler(feature_range=(0.0001, 0.9999))
        
        x_inv = matrix_inverse(x, True)  # inverse of the input features matrix
        
        for j in trange(l_max, file=sys.stdout):
            #Step1
            if j == 0:
                #intialize the input weights and bais randomly
                af = np.random.normal(size = (features, d))
                bf = np.random.normal(size = (1, 1))
                # calculate the representation
                h = self.act(np.dot(x, af) + bf)
            #Step 1 is done
            
            #Step4
            else:
                pj_norm = self.act_inv(p_scaler.fit_transform(Pj) + 0j).real
            
                af = np.dot(x_inv, pj_norm)
                bf = mean_squared_error(pj_norm, Pj, squared=False)
                
                h = self.act(np.dot(x, af) + bf)
                h += p_scaler.inverse_transform(h)
            #Step 4 is done
  
            self.af.append(af)
            self.bf.append(bf)
            self.scalers.append(p_scaler)
            
            #Step2
            y_norm = self.act_inv(y_scaler.fit_transform(y) + 0j).real
            ah = np.dot(matrix_inverse(h, True, coefficient), y_norm)
            bh = mean_squared_error(np.dot(h, ah), y_norm, squared=False)
            #Step 2 is done
            
            #Step3
            elm_h = self.act(np.dot(h, ah) + bh)

            ej = y - y_scaler.inverse_transform(elm_h)
            e_norm = self.act_inv(y_scaler.transform(ej) + 0j).real
            
            Pj = np.dot(e_norm, matrix_inverse(ah, True))
            #Step 3 is done


    #Retrieves the representation for the input data using the trained model parameters
    def transform(self, x):
        if len(self.af) == 0:
            raise 'There is no feature mapping model'
            
        feature_data = None
        for i in range(len(self.af)):
            h = self.act(np.dot(x, self.af[i]) + self.bf[i])  
            if feature_data is None:
                feature_data = h
            else:
                h = self.scalers[i].inverse_transform(h)
                feature_data += h
        return feature_data
    
    # Trains the model and get the represention for the given data
    def fit_transform(self, x, y, d, l_max, act_fn):
        self.fit(x, y, d, l_max, act_fn)
        return self.transform(x)
        

In [10]:
x_train_m = x_train
y_train_m = y_train
x_test_m = x_test
y_test_m = y_test

# Create an object
m = MSNN()
# Train the model
m.fit(x = x_train_m, y = y_train_m, d = 41, l_max = 2, act_fn = 'sigmoid', coefficient = 20)
# Retrive the new representations form the trained model
x_train_new = m.transform(x_train_m)
x_test_new = m.transform(x_test_m)

# Normalize the new input features
mm_scaler = MinMaxScaler()
x_train_norm = mm_scaler.fit_transform(x_train_new)
x_test_norm = mm_scaler.transform(x_test_new)

# ELM is used for the classification
elm = ELM(hidden_nodes = 1000, activation = 'relu')
_, train_acc = elm.fit(x_train_norm, y_train)
_, test_acc = elm.evaluate(x_test_norm, y_test)
print('\tTraining Accuracy : ', train_acc)
print('\tTesting Accuracy : ', test_acc)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.82it/s]
	Training Accuracy :  99.87
	Testing Accuracy :  8.33
